In [1]:
!pip install --upgrade lookerapi

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
Requirement already up-to-date: lookerapi in /anaconda2/lib/python2.7/site-packages (3.0.0)


In [5]:
import requests, json, logging, time, urllib
import pandas as pd
from io import StringIO
import io as stringIOModule
from datetime import datetime

In [6]:
# Set up logging to catch errors, etc...

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.FileHandler(str(time.strftime("%d_%m_%Y")) +"_looker_API_Calls" + ".log")
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.info('Testing Logs')

# Configure Credentials 


For the purpose of this exercise, we made the credentials available in this python notebook... 


In a production environment, you could setup credentials in a seperate file called looker-config and read in credentials here. 
Sample credentails might look like the following: 
#### Connecting to the Looker API 
[looker-creds]<br/>
api_endpoint: https://<instance>.looker.com:19999/api/3.0 <br/>
login_url: https://<instance>.looker.com:19999/login <br/>
client_id: <Client ID> <br/>
client_secret: <Secret key> <br/>

#### Connecting to a Database
Example creds for Redshift might look like this: <br/>
[redshift] <br/>
username: my_username <br/>
password: my_pwd <br/>
host_name: my_hostname <br/>
port: port_num <br/>
db_name: my_dbname <br/>

In [7]:
class LookerAPI(object):
    """Class that contains methods and variables related to looker API authentication"""
    def __init__(self, api_info):
        self.api_endpoint = api_info['api_endpoint']
        self.client_secret = api_info['client_secret']
        self.client_id = api_info['client_id']
        self.login_endpoint = api_info['login_url']
        print(self.login_endpoint)
        
    def login(self):
        "login to looker API"
        try:
            auth_data = {'client_id':self.client_id, 'client_secret':self.client_secret}
            r = requests.post( self.login_endpoint,data=auth_data) # error handle here
            json_auth = json.loads(r.text)['access_token']
            return json_auth
        except requests.exceptions.RequestException as e:
            logger.error(e)

    def run_look(self, look_id, json_auth,return_format='csv'):
        "run looks"
        try:
            look_run_url = self.api_endpoint + '/looks/{0}/run/{1}'.format(look_id,return_format)
            #r = requests.get(look_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(look_run_url + '?' + 'access_token=' + json_auth)
            return r.text
        except requests.exceptions.RequestException as e:
            logger.error(e)
            
    def run_query(self, query_id, json_auth, return_format='csv'):
        "run query"
        try:
            query_run_url = self.api_endpoint + '/queries/{0}/run/{1}'.format(query_id,return_format)
            #r = requests.get(query_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(query_run_url + '?' + 'access_token=' + json_auth)
            return r.text
        except requests.exceptions.RequestException as e:
            logger.error(e)
            
    def run_query_slug(self, query_slug, json_auth):
        "run query"
        try:
            query_slug_run_url = self.api_endpoint + '/queries/slug/{0}'.format(query_slug)
            #r = requests.get(query_run_url, headers={'Authorization': "token " + json_auth})
            r = requests.get(query_slug_run_url + '?' + 'access_token=' + json_auth)
            qid=json.loads(r.text)["id"]
            print("Query_id: " + str(qid))
            return LookerAPI.run_query(self, qid, json_auth)
        except requests.exceptions.RequestException as e:
            logger.error(e)

In [20]:
# Set API Credentials for Your Look Instance

creds = {
    'api_endpoint' : 'https://profservices.dev.looker.com:19999/api/3.0',
    'login_url': 'https://profservices.dev.looker.com:19999/login',
    'client_id' : '<Your Client ID>', 
    'client_secret': '<Youe client secret>'
}

demo = LookerAPI(creds)

# Login to Instance - Return token to be used in subsequent calls.
json_auth = demo.login()

#print('Token: ' + json_auth)


# Get Results of Look and Query ID (QID)
look_response = demo.run_look(292,json_auth)
print('###### Printing the Look ###########')
print(look_response)

query_response = demo.run_query(53213, json_auth, return_format='csv')
print('###### Printing the Query Results through Query_ID ###########')
print(query_response)





https://profservices.dev.looker.com:19999/login
###### Printing the Look ###########
Products Category,Products Total Cost
Outerwear & Coats,42762.249566
Jeans,39780.728724
Sweaters,38769.229926
Fashion Hoodies & Sweatshirts,35371.796679
Shorts,29869.195966
Suits & Sport Coats,29692.746962
Dresses,28794.617029
Tops & Tees,28167.180032
Swim,27902.696162
Sleep & Lounge,21096.64692
Active,19539.922708
Accessories,17382.989449
Blazers & Jackets,16367.546703
Pants,15375.514387
Pants & Capris,10798.08969
Intimates,10586.897036
Suits,9106.845037
Plus,8255.791606
Leggings,6853.144542
Skirts,6626.249785
Underwear,5478.529214
Maternity,5060.804544
Socks,4045.013272
Socks & Hosiery,2325.51955
Jumpsuits & Rompers,2178.494167
Clothing Sets,1219.560783

###### Printing the Query Results through Query_ID ###########
Products Category,Products Total Cost
Outerwear & Coats,42762.249566
Jeans,39780.728724
Sweaters,38769.229926
Fashion Hoodies & Sweatshirts,35371.796679
Shorts,29869.195966
Suits & Sport 

In [21]:
# # Convert Query Response from CSV to a dateframe
records = pd.read_csv(StringIO(query_response))
records.head(10)

,Products Category,Products Total Cost
0,Outerwear & Coats,42762.249566
1,Jeans,39780.728724
2,Sweaters,38769.229926
3,Fashion Hoodies & Sweatshirts,35371.796679
4,Shorts,29869.195966
5,Suits & Sport Coats,29692.746962
6,Dresses,28794.617029
7,Tops & Tees,28167.180032
8,Swim,27902.696162
9,Sleep & Lounge,21096.646920
